In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#라이브러리와 app 데이터 세트 로딩

import numpy as np
import pandas as pd
import gc
import time

%matplotlib inline

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

**application_train.csv파일과 application_test.csv파일 불러오기**

In [ ]:
app_train=pd.read_csv('../input/home-credit-default-risk/application_train.csv')
app_test=pd.read_csv('../input/home-credit-default-risk/application_test.csv')

# **application_train에 대한 EDA진행**

In [ ]:
app_train.isnull().sum()

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
app_train.info()

**연속형 숫자 feature들에 대해서 TARGET값이 각각 0과 1일때의 Histogram 시각화**

In [ ]:
#히스토그램 시각화 정의

def show_hist_by_target(df,columns):
    cond_1=(df['TARGET']==1)
    cond_0=(df['TARGET']==0)
    
    for column in columns:
        fig,ax=plt.subplots(figsize=(12,4),nrows=1,ncols=2,squeeze=False)
        sns.violinplot(x='TARGET',y=column,data=df,ax=ax[0][0])
        sns.distplot(df[cond_1][column],label='1',color='red',ax=ax[0][1])
        sns.distplot(df[cond_0][column],label='0',color='blue',ax=ax[0][1])


In [ ]:
columns=['DAYS_LAST_PHONE_CHANGE']

show_hist_by_target(app_train, columns)

**credit card balance 데이터에서 새로운 feature 만들어서 application train, application test와 previous application 결합하여 모델링**

In [ ]:
# 데이터 확인
apps = pd.concat([app_train, app_test])
print("Size of application_train data", app_train.shape)
print("Size of application_test data", app_test.shape)
print("Size of apps", apps.shape)

In [ ]:
# 데이터 가져오기
prev = pd.read_csv('../input/home-credit-default-risk/previous_application.csv')
card = pd.read_csv('../input/home-credit-default-risk/credit_card_balance.csv')


In [ ]:
# 가져온 데이터 확인
print("Size of application_train data", prev.shape)
print("Size of application_test data", card.shape)

In [ ]:
#card 데이터 확인
card.head(15)

In [ ]:
#ID 378907의 신용 카드 정보 확인
card[card['SK_ID_CURR']==378907]

# **EDA-TARGET 유형에 따라 Card 데이터(credit card balance) 비교**



In [ ]:
# application_train 데이터와 card 데이터 merege
app_card = card.merge(app_train[['SK_ID_CURR', 'TARGET']], left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='inner')
app_card.shape

In [ ]:
f,ax=plt.subplots(1,2,figsize=(12,6))
app_train.TARGET.value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Distribution of Target')
ax[0].set_ylabel('')
sns.countplot('TARGET',data=app_train,ax=ax[1])
ax[1].set_title('Target count')
plt.show()

In [ ]:
# null data 정의
def nulldata(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    ms=pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    ms= ms[ms["Percent"] > 0]
    f,ax =plt.subplots(figsize=(15,10))
    plt.xticks(rotation='90')
    fig=sns.barplot(ms.index, ms["Percent"],color="green",alpha=0.8)
    plt.xlabel('Features', fontsize=15)
    plt.ylabel('Percent of null values', fontsize=15)
    plt.title('Percent null data by feature', fontsize=15)
    return ms

In [ ]:
#card null data 퍼센트 확인
nulldata(card)

In [ ]:
# Numerical Features(숫자형 피처) 과 Categorical Features(범주/object 피처) 구분 함수

def type_features(data):
    categorical_features = data.select_dtypes(include = ["object"]).columns
    numerical_features = data.select_dtypes(exclude = ["object"]).columns
    print( "categorical_features :",categorical_features)
    print('-----'*40)
    print("numerical_features:",numerical_features)

In [ ]:
# card 데이터의 숫자형 피처와 범주형 피처 구분
type_features(card)

In [ ]:
# 시각화 함수
def plot_re(df,t1='',t2=''):
    f,ax=plt.subplots(1,2,figsize=(12,8))
    df[[t1,t2]].groupby([t1]).count().plot.bar(ax=ax[0],color='Blue')
    ax[0].set_title('count of customer on '+t1)
    sns.countplot(t1,hue=t2,data=df,ax=ax[1],palette="spring")
    ax[1].set_title(t1+': Target 0 vs Target 1')
    # Rotate x-labels
    plt.xticks(rotation=-90)
    a=plt.show()
    return a

In [ ]:
plot_re(app_card,'NAME_CONTRACT_STATUS','TARGET')
#이전 신용에 대한 계약 상태가 

# Numerical Feature 시각화

In [ ]:
# card 데이터 correlated features 분석

corrmat = app_card.corr()
top_corr_features = corrmat.index[abs(corrmat["TARGET"])>=0.03]
plt.figure(figsize=(12,8))
g = sns.heatmap(app_card[top_corr_features].corr(),annot=True,cmap="Oranges")

In [ ]:
f, ax = plt.subplots(2,3,figsize=(13, 10))

# MONTHS_BALANCE: 신청일 기준 잔액 월(-1은 가장 최근 잔액 월 의미)
sns.distplot(card.MONTHS_BALANCE.dropna(), kde=True, color="b",
             ax=ax[0,0]).set_title('MONTHS_BALANCE Distribution')


# AMT_DRAWINGS_CURRENT: 월별 인출 금액
sns.distplot(card.AMT_DRAWINGS_CURRENT.dropna(), kde=True, color="g", 
             ax=ax[0,1]).set_title('AMT_DRAWINGS_CURRENT Distribution')

# AMT_PAYMENT_CURRENT: 월별 고객이 납부한 금액
sns.distplot(card.AMT_PAYMENT_CURRENT.dropna(), kde=True, color="b",
             ax=ax[0,2]).set_title('AMT_PAYMENT_CURRENT Distribution')


# AMT_PAYMENT_TOTAL_CURRENT: 고객이 최종 납부한 금액
sns.distplot(card.AMT_PAYMENT_TOTAL_CURRENT.dropna(), kde=True, color="r", 
             ax=ax[1,0]).set_title('AMT_PAYMENT_TOTAL_CURRENT Distribution')


# AMT_RECIVABLE: 대출 원금 회수 금액
sns.distplot(card.AMT_RECIVABLE.dropna(), kde=True, color="g", 
             ax=ax[1,1]).set_title('AMT_RECIVABLE Distribution')


# SK_DPD: 연체 일자
sns.distplot(card.SK_DPD.dropna(), kde=True, color="r", 
             ax=ax[1,2]).set_title('SK_DPD Distribution')


In [ ]:
# NAME_CONTRACT_STATUS 계약 상태
# AMT_CREDIT_LIMIT_ACTUAL 월별 카드 허용한도

sns.stripplot(x="AMT_CREDIT_LIMIT_ACTUAL", y="NAME_CONTRACT_STATUS", data=app_card)
plt.show()

# 결과: 계약 상태별 월별 카드 허용한도 확인 가능 -> 계약 상태가 활성화된 경우, 월별 카드 허용한도가 다른 상황에 비해 대체로 높음을 확인

In [ ]:
# 허용가능한 연체 일자와 납부 횟수의 상관관계
# CNT_INSTALMENT_MATURE_CUM 납부 횟수
# SK_DPD_DEF 허용가능한 연체 일자 

sns.jointplot(x="CNT_INSTALMENT_MATURE_CUM", y="SK_DPD_DEF", data=app_card)
plt.show()

# 허용 가능한 연체 일자가 높을수록 납부 횟수의 수가 많아질 것이라고 예상하였는데, 허용 기간이 어떠하든 간에 대체로 납부 횟수는 적다.
# 허용 가능한 연체 일자가 적을때 사람들의 납부 횟수가 더 많다.

# **Credit Card balance Data Feature Engineering**


In [ ]:
# 고객 당 월별 인출 금액

card_fe1= card

AMT_DRAWINGS_CURRENT_AMOUNT = card_fe1[['SK_ID_CURR', 'AMT_DRAWINGS_CURRENT']].groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_CURRENT'].nunique().reset_index().rename(index=str, columns={'AMT_DRAWINGS_CURRENT': 'AMT_DRAWINGS_CURRENT_AMOUNT'})
card_fe1 = card_fe1.merge(AMT_DRAWINGS_CURRENT_AMOUNT, on = ['SK_ID_CURR'], how = 'left')
print(card_fe1.shape)
card_fe1.head()


In [ ]:
# 고객 당 월별 인출 횟수

CNT_DRAWINGS_CURRENT_NUMBER = card_fe1[['SK_ID_CURR', 'CNT_DRAWINGS_CURRENT']].groupby(by = ['SK_ID_CURR'])['CNT_DRAWINGS_CURRENT'].nunique().reset_index().rename(index=str, columns={'CNT_DRAWINGS_CURRENT': 'CNT_DRAWINGS_CURRENT_NUMBER'})
card_fe1 = card_fe1.merge(CNT_DRAWINGS_CURRENT_NUMBER, on = ['SK_ID_CURR'], how = 'left')
print(card_fe1.shape)
card_fe1.head()

In [ ]:
# 고객 당 평균 인출 금액= 고객 당 월별 인출 금액 / 고객 당 월별 인출 횟수
# --> 고객의 평균 인출 금액을 파악해 볼 수 있음

card_fe1['AVERAGE_WITHDRAWAL_AMOUNT'] = card_fe1['AMT_DRAWINGS_CURRENT_AMOUNT']/card_fe1['CNT_DRAWINGS_CURRENT_NUMBER']

print(card_fe1.shape)
card_fe1.head(15)

In [ ]:
# card 데이터의 계약 완료된(Completed) 비율 


card_fe1['NAME_CONTRACT_STATUS_COMPLETED'] = card_fe1['NAME_CONTRACT_STATUS']

# completed 인 경우에는 y에는 0, 나머지는 Y=1 
def f(x):
    if x == 'Completed':
        y = 0
    else:
        y = 1    
    return y

card_fe1['NAME_CONTRACT_STATUS_COMPLETED'] = card_fe1.apply(lambda x: f(x.NAME_CONTRACT_STATUS), axis = 1)

# CUSTOMER 당 계약 완료된 상태인 평균 신용 카드 잔액 계산
grp = card_fe1.groupby(by = ['SK_ID_CURR'])['NAME_CONTRACT_STATUS_COMPLETED'].mean().reset_index().rename(index=str, columns={'NAME_CONTRACT_STATUS_COMPLETED':'ACTIVE_LOANS_PERCENTAGE'})
card_fe1= card_fe1.merge(grp, on = ['SK_ID_CURR'], how = 'left')

print(card_fe1.shape)

In [ ]:
# 고객 당 최종 납부 금액

card_fe2=card

AMT_PAYMENT_TOTAL_CURRENT_AMOUNT = card_fe2[['SK_ID_CURR', 'AMT_PAYMENT_TOTAL_CURRENT']].groupby(by = ['SK_ID_CURR'])['AMT_PAYMENT_TOTAL_CURRENT'].nunique().reset_index().rename(index=str, columns={'AMT_PAYMENT_TOTAL_CURRENT': 'AMT_PAYMENT_TOTAL_CURRENT_AMOUNT'})
card_fe2 = card_fe2.merge(AMT_PAYMENT_TOTAL_CURRENT_AMOUNT, on = ['SK_ID_CURR'], how = 'left')
print(card_fe2.shape)
card_fe2.head()

In [ ]:
# 고객 당 납부 횟수

CNT_INSTALMENT_MATURE_CUM_NUMBER = card_fe1[['SK_ID_CURR', 'CNT_INSTALMENT_MATURE_CUM']].groupby(by = ['SK_ID_CURR'])['CNT_INSTALMENT_MATURE_CUM'].nunique().reset_index().rename(index=str, columns={'CNT_INSTALMENT_MATURE_CUM': 'CNT_INSTALMENT_MATURE_CUM_NUMBER'})
card_fe2 = card_fe2.merge(CNT_INSTALMENT_MATURE_CUM_NUMBER, on = ['SK_ID_CURR'], how = 'left')
print(card_fe2.shape)
card_fe2.head()

In [ ]:
# 고객 당 횟수 별 평균 납부 금액= 고객 당 최종 납부 금액 / 고객 당 납부 횟수
# --> 고객이 납부 할때 평균 납부 금액을 알 수 있음

card_fe2['AVERAGE_PAYMENT_AMOUNT'] = card_fe2['AMT_PAYMENT_TOTAL_CURRENT_AMOUNT']/card_fe2['CNT_INSTALMENT_MATURE_CUM_NUMBER']

print(card_fe2.shape)
card_fe2.head(15)

# **application_train, credit_card_balance 데이터 결합 및 모델링**

In [ ]:
#원래 데이터에서 천만개 데이터만 저장
card_fe1[0:1000000].to_csv('card_fe3.csv',index=False)
card_fe2[0:1000000].to_csv('card_fe4.csv',index=False)
app_train.to_csv('apptraintest.csv',index=False)

In [ ]:
#필요한 데이터 읽기
apptraintest=pd.read_csv('apptraintest.csv')
card_pre_1=pd.read_csv('card_fe3.csv')
card_pre_2=pd.read_csv('card_fe4.csv')

In [ ]:
# 데이터 확인
print("Size of application_train + test data",apptraintest.shape)
print("Size of credit_card_balance_data1",card_pre_1.shape)
print("Size of credit_card_balance_data1 data",card_pre_2.shape)

In [ ]:
#가공된 최종 데이터 세트 생성

apps_all=apptraintest.merge(card_pre_1, on='SK_ID_CURR',how='left')
apps_all=apptraintest.merge(card_pre_2, on='SK_ID_CURR',how='left')
apps_all.shape

In [ ]:
#apps_all 데이터 정보 확인
apps_all.info()

In [ ]:
#label encoding
object_columns=apps_all.dtypes[apps_all.dtypes=='object'].index.tolist()
for column in object_columns:
    apps_all[column]=pd.factorize(apps_all[column])[0]

In [ ]:
#학습 데이터와 테스트 데이터 다시 분리
apps_all_train=apps_all[~apps_all['TARGET'].isnull()]
apps_all_test=apps_all[apps_all['TARGET'].isnull()]

apps_all_test=apps_all_test.drop('TARGET',axis=1)

apps_all_test.columns.tolist()


In [ ]:
#학습 데이터를 검증 데이터로 분리하고 LGBM Classifier로 학습 수행
from sklearn.model_selection import train_test_split

ftr_app=apps_all_train.drop(['SK_ID_CURR','TARGET'],axis=1)
target_app=apps_all_train['TARGET']

train_x, valid_x, train_y, valid_y=train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
train_x.shape, valid_x.shape

In [ ]:
pip install lightgbm

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
clf=LGBMClassifier(
    nthread=4,
    n_estimators=10000,
    learning_rate=0.02,
    num_leaves=32,
    colsample_bytree=0.9497036,
    subsample=0.8715623,
    max_depth=8,
    reg_alpha=0.04,
    reg_lambada=0.073,
    min_split_gain=0.0222415,
    min_child_weight=40,
    silent=-1,
    verbose=-1)
clf.fit(train_x,train_y,eval_set=[(train_x,train_y),(valid_x,valid_y)],eval_metric='auc',verbose=100,
       early_stopping_rounds=50)

In [ ]:
clf.predict_proba(app_test.drop(['SK_ID_CURR'],axis=1))

In [ ]:
#학습된 Classifier를 이용하여 테스트 데이터를 예측하고 결과를 Kaggle로 submit
pred=clf.predict_proba(app_test.drop(['SK_ID_CURR'],axis=1))[:,1]
app_test['TARGET']=pred
app_test['TARGET'].head()


In [ ]:
app_test[['SK_ID_CURR','TARGET']].to.csv('BigData_project_01.csv',index=False)